In [1]:
import txjupyter
reactor = txjupyter.start('tcp:port=8080')

In [2]:
from klein import route
@route('/')
def home(request):
    return 'Hello, world!'

In [3]:
@route('/foo')
def froop(request):
    return 'Goodbye!'